In [2]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --inplace your_notebook.ipynb

[NbConvertApp] WARNING | pattern 'your_notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
 

In [ ]:
!pip install langchain langchain-openai langchain-community langchain-text-splitters langchain-unstructured \
faiss-cpu openai tiktoken unstructured beautifulsoup4 python-dotenv
!pip install wikipedia
!pip install arxiv
!pip install huggingface_hub langchain langchain_community

In [ ]:
!pip install transformers accelerate bitsandbytes

from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

# Hugging Face pipeline
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    max_length=512
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=generator)

# Test
print(llm("What is Machine Learning?"))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-585205398.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)
/tmp/ipython-input-585205398.py:17: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm("What is Machine Learning?"))


machine learning machine learning is the use of artificial intelligence to learn from data.


In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain.agents import initialize_agent, Tool

wiki = WikipediaAPIWrapper()
arxiv = ArxivAPIWrapper()

tools = [
    Tool(name="Wikipedia", func=wiki.run, description="Search Wikipedia"),
    Tool(name="Arxiv", func=arxiv.run, description="Search Arxiv for research papers")
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)



/tmp/ipython-input-2999321853.py:12: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [ ]:
#setup documents & text splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Example documents (you can replace with PDFs, Wikipedia, etc.)
docs = [
    """India is the seventh-largest country by land area, the most populous country,
    and the most populous democracy in the world. It is in South Asia and shares
    borders with Pakistan, China, Nepal, Bhutan, Bangladesh, and Myanmar."""
]

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)
documents = text_splitter.create_documents(docs)

print(f"Number of chunks: {len(documents)}")
print(documents[0])



Number of chunks: 2
page_content='India is the seventh-largest country by land area, the most populous country,
    and the most populous democracy in the world. It is in South Asia and shares'


In [ ]:
#converting text into embeddings

from langchain_community.embeddings import HuggingFaceEmbeddings

# Use a small embedding model from Hugging Face
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Test embedding
print(len(embedding_model.embed_query("India is a country.")))


/tmp/ipython-input-2940941124.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

384


In [ ]:

from langchain.chains import RetrievalQA

# RAG pipeline
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=db.as_retriever()
)

# Ask a question
response = qa.run("Tell me about India's neighbors.")
print("\nAnswer:", response)



Answer: shares borders with Pakistan, China, Nepal, Bhutan, Bangladesh, and Myanmar


In [ ]:
#storing embeddings into vector database(FAISS)
from langchain_community.vectorstores import FAISS

# Create FAISS index
db = FAISS.from_documents(documents, embedding_model)

# Test retrieval
query = "Which countries share a border with India?"
retrieved_docs = db.similarity_search(query, k=2)

for doc in retrieved_docs:
    print(doc.page_content)


borders with Pakistan, China, Nepal, Bhutan, Bangladesh, and Myanmar.
India is the seventh-largest country by land area, the most populous country,
    and the most populous democracy in the world. It is in South Asia and shares


In [ ]:
#Loading real world docs into vector database
# Install required packages
!pip install faiss-cpu sentence-transformers langchain langchain-community pypdf


In [ ]:
#loading and uploading pdf
#here i am using a pdf about india which i downloaded from official NCERT website
from google.colab import files

uploaded = files.upload()  # choose INDIA_RAG.pdf from your computer



Saving INDIA_RAG.pdf to INDIA_RAG (1).pdf


In [ ]:
loader = PyPDFLoader("/content/INDIA_RAG.pdf")
documents = loader.load()


In [ ]:
#Splitting docs into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,     # max tokens per chunk
    chunk_overlap=200    # overlap between chunks for context
)
docs = text_splitter.split_documents(documents)

print(f"Total chunks: {len(docs)}")
print(docs[0].page_content[:300])  # preview first chunk


Total chunks: 17
75
5 ― India, That Is Bharat
India, That Is Bharat
CHAPTER5 
In India at a very early time the spiritual and cultural unity was 
made complete and became the very stuff of the life of all this great 
surge of humanity between the Himalayas and the two seas.
— Sri Aurobindo
Fig. 5.1. A scene from rur


In [ ]:
!pip install -U langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# Use a small but good embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
from langchain_community.vectorstores import FAISS

# Store all document chunks
vectorstore = FAISS.from_documents(docs, embeddings)

# Save for later use
vectorstore.save_local("faiss_index")


In [ ]:
from langchain.chains import RetrievalQA

# Reload FAISS (optional, if saved earlier)
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

# Build retrieval-based QA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff"
)

# Ask a question
query = "What is Machine Learning according to this PDF?"
result = qa.run(query)
print(result)


Token indices sequence length is longer than the specified maximum sequence length for this model (523 > 512). Running this sequence through the model will result in indexing errors


a physical map of the Indian Subcontinent, with some of its rivers


In [ ]:
#making this conversational chatbot
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Memory to store chat history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Conversational chain
chatbot = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory
)

# Simple chat loop
print("🤖 Chatbot ready! Type 'exit' to stop.\n")
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye 👋")
        break
    result = chatbot({"question": query})
    print("Chatbot:", result["answer"])


/tmp/ipython-input-4237784841.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


🤖 Chatbot ready! Type 'exit' to stop.

You: tell me something about ancient india


/tmp/ipython-input-4237784841.py:21: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chatbot({"question": query})


Chatbot: Today, the India we know is a modern nation, with defined borders, defined states and a known population. However , it was very different 500 years ago, 2,000 years ago or even 5,000 years ago.
You: exit
Chatbot: Goodbye 👋
